In [2]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

## Prepare test data

In [ ]:
# !git clone https://github.com/hbcbh1999/recaptcha-dataset.git

Cloning into 'recaptcha-dataset'...
remote: Enumerating objects: 11754, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 11754 (delta 5), reused 1 (delta 0), pack-reused 11742
Receiving objects: 100% (11754/11754), 395.64 MiB | 22.89 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Updating files: 100% (11779/11779), done.


### Preprocessing

In [50]:
image_dir = "C:/Users/user/Desktop/2025 컴퓨터비전/Challange1/testset"
image_list = []

for fname in sorted(os.listdir(image_dir)):
    if fname.endswith(".png"):
        img_path = os.path.join(image_dir, fname)
        image = cv2.imread(img_path)

        if image is None:
            print(f"이미지 로딩 실패: {img_path}")
            continue 
        
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
        image_list.append((fname, gray))

## Feature extraction

In [38]:
def norm_hist(hist):
    # Normalize the histogram
    hist = hist.astype('float')
    hist /= hist.sum()
    return hist

### LBP

In [51]:
from skimage.feature import local_binary_pattern

lbp_features = []

# LBP
for fname, gray in image_list:
    lbp = local_binary_pattern(gray, P=8, R=1)

    hist_lbp, bin_lbp = np.histogram(lbp.ravel(), bins=64, range=(0, 256))
    hist_lbp = norm_hist(hist_lbp)    # 64-d
    lbp_features.append((fname, hist_lbp))

### Law's texture

In [53]:
# Law's texture
from scipy import signal as sg

def laws_texture(gray):
    (rows, cols) = gray.shape[:2]

    smooth_kernel = (1/25)*np.ones((5,5))
    gray_smooth = sg.convolve(gray, smooth_kernel,"same")
    gray_processed = np.abs(gray - gray_smooth)

    filter_vectors = np.array([[ 1,  4,  6,  4, 1],    # L5
                               [-1, -2,  0,  2, 1],    # E5
                               [-1,  0,  2,  0, 1],    # S5
                               [ 1, -4,  6, -4, 1]])   # R5

    # 0:L5L5, 1:L5E5, 2:L5S5, 3:L5R5,
    # 4:E5L5, 5:E5E5, 6:E5S5, 7:E5R5,
    # 8:S5L5, 9:S5E5, 10:S5S5, 11:S5R5,
    # 12:R5L5, 13:R5E5, 14:R5S5, 15:R5R5
    filters = list()
    for i in range(4):
        for j in range(4):
            filters.append(np.matmul(filter_vectors[i][:].reshape(5,1),
                                     filter_vectors[j][:].reshape(1,5)))

    conv_maps = np.zeros((rows, cols,16))
    for i in range(len(filters)):
        conv_maps[:, :, i] = sg.convolve(gray_processed,
                                         filters[i],'same')

    texture_maps = list()
    texture_maps.append((conv_maps[:, :, 1]+conv_maps[:, :, 4])//2)     # L5E5 / E5L5
    texture_maps.append((conv_maps[:, :, 2]+conv_maps[:, :, 8])//2)     # L5S5 / S5L5
    texture_maps.append((conv_maps[:, :, 3]+conv_maps[:, :, 12])//2)    # L5R5 / R5L5
    texture_maps.append((conv_maps[:, :, 7]+conv_maps[:, :, 13])//2)    # E5R5 / R5E5
    texture_maps.append((conv_maps[:, :, 6]+conv_maps[:, :, 9])//2)     # E5S5 / S5E5
    texture_maps.append((conv_maps[:, :, 11]+conv_maps[:, :, 14])//2)   # S5R5 / R5S5
    texture_maps.append(conv_maps[:, :, 10])                            # S5S5
    texture_maps.append(conv_maps[:, :, 5])                             # E5E5
    texture_maps.append(conv_maps[:, :, 15])                            # R5R5
    texture_maps.append(conv_maps[:, :, 0])                             # L5L5 (use to norm TEM)

    TEM = list()
    for i in range(9):
        TEM.append(np.abs(texture_maps[i]).sum() / np.abs(texture_maps[9]).sum())

    return TEM


laws_feature = []

for fname, gray in image_list:
    laws = laws_texture(gray)    # 9-d
    laws_feature.append((fname, np.array(laws)))

### Load .npy Files

In [54]:
used_features = ['lbp', 'laws']
feature_dir = "C:/Users/user/Desktop/2025 컴퓨터비전/Challange1/prepared"

X_train = np.load(os.path.join(feature_dir, "X_train_lbp_laws.npy"))
y_train = np.load(os.path.join(feature_dir, "y_train_lbp_laws.npy"))

### Combine feature vectors

In [55]:
test_features = []
test_filenames = []

for (fname1, lbp), (fname2, laws) in zip(lbp_features, laws_feature):
    assert fname1 == fname2, f"{fname1} != {fname2}"
    combined = np.concatenate([lbp, laws])
    test_features.append(combined)
    test_filenames.append(fname1)

X_test = np.array(test_features)

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

classifier = KNeighborsClassifier(n_neighbors = 3)

classifier.fit(X_train, y_train)
predict_labels = classifier.predict(X_test)

### Task 1: Classification

In [67]:
predict_labels = classifier.predict(test_features)
print(predict_labels)    # array(100)

['Traffic Light' 'Car' 'Crosswalk' 'Car' 'Car' 'Bus' 'Car' 'Bus']


In [68]:
import csv

# with open('c1_t1_a1.csv','w') as file :
#     write = csv.writer(file)
#     for i, predict_label in enumerate(predict_labels):
#         write.writerow([f'query{i+1:03}.png', predict_label])

with open('c1_t1_a2.csv','w') as file :
    writer = csv.writer(file)
    for fname, pred in zip(test_filenames, predict_labels):
        writer.writerow([fname, pred])

### Task 2: Retrieval

In [63]:
neigh_ind = classifier.kneighbors(X=test_features, n_neighbors=10, return_distance=False) # Top-10 results
neigh_labels = np.array(y_train)[neigh_ind]

In [31]:
print(neigh_labels)    # array(100x10)

[['Palm' 'Palm' 'Traffic Light' 'Palm' 'Palm' 'Car' 'Car' 'Palm' 'Car'
  'Crosswalk']
 ['Crosswalk' 'Crosswalk' 'Crosswalk' 'Crosswalk' 'Hydrant' 'Car'
  'Crosswalk' 'Crosswalk' 'Palm' 'Traffic Light']
 ['Bicycle' 'Palm' 'Hydrant' 'Crosswalk' 'Crosswalk' 'Crosswalk'
  'Crosswalk' 'Palm' 'Crosswalk' 'Palm']
 ['Hydrant' 'Palm' 'Hydrant' 'Crosswalk' 'Hydrant' 'Palm' 'Traffic Light'
  'Hydrant' 'Bus' 'Crosswalk']]


In [64]:
import csv

with open('c1_t2_a1.csv','w') as file :
    write = csv.writer(file)
    for i, neigh_label in enumerate(neigh_labels):
        write.writerow([f'query{i+1:03}.png'] + list(neigh_label))